In [ ]:
from app.backend.core.processor import FileProcessor

processor = FileProcessor()
content = processor.extract_from_file_sync("data/ZacharyBiery_CV_2024.docx")
print(content)

RuntimeError: asyncio.run() cannot be called from a running event loop